In [13]:
from IPython import get_ipython
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("/content/spam.csv", encoding='latin-1')

df.head()

df.shape

df.isnull().sum()

df=df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])

df=df.rename(columns={"v2":"message","v1":"labels"})

df.head()

df.duplicated()

df.info()

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
ps=WordNetLemmatizer()
nltk.download('wordnet')

corpus=[]
for i in range (0,len(df['message'])):
  review=re.sub('[^a-z A-Z]','',df["message"][i])
  review=review.lower()
  review=review.split()
  review=[ps.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
  review=' '.join(review)
  corpus.append(review)

y=pd.get_dummies(df['labels'])
y=y.iloc[:,1].values
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=42, stratify=y)

#use Bow method
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500,binary=True)
X_train=cv.fit_transform(X_train).toarray()
X_test=cv.transform(X_test).toarray()








from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # Import metrics


models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(),
    "Support Vector Machine": SVC(),
    "Random Forest": RandomForestClassifier(),

}

# Modify the evaluate_models function to handle the single-column target variable
def evaluate_models(X_train, X_test, y_train, y_test, models: dict):
    """
    Trains and evaluates multiple models.

    Parameters:
        X_train, X_test: your data splits
        y_train, y_test: your single-column numeric target variable splits (e.g., 0/1)
        models (dict): A dictionary of models e.g., {'Logistic Regression': LogisticRegression()}

    Returns:
        results_df: A DataFrame showing the scores of each model
    """
    results = []

    # y_train and y_test are already single-column Series after the change above
    # No need to select 'spam' column here anymore

    for name, model in models.items():
        # Train the model using the single-column numeric labels
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        results.append({
            'Model': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            # Use pos_label=1 since 'spam' is encoded as 1
            # Specify the labels that can appear in y_true and y_pred
            'Precision': precision_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0, labels=[0, 1]),
            'Recall': recall_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0, labels=[0, 1]),
            'F1 Score': f1_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0, labels=[0, 1])
        })

    results_df = pd.DataFrame(results).sort_values(by='F1 Score', ascending=False)
    return results_df

results_df = evaluate_models(X_train, X_test, y_train, y_test, models)
print(results_df)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   labels   5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                    Model  Accuracy  Precision    Recall  F1 Score
1     Logistic Regression  0.979372   1.000000  0.845638  0.916364
2  Support Vector Machine  0.976682   1.000000  0.825503  0.904412
3           Random Forest  0.974888   0.984000  0.825503  0.897810
0             Naive Bayes  0.973094   0.916084  0.879195  0.897260


In [14]:
#using tfidf
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=42, stratify=y)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=2500,ngram_range=(1,3))
X_train=tfidf.fit_transform(X_train).toarray()
X_test=tfidf.transform(X_test).toarray()

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # Import metrics


models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(),
    "Support Vector Machine": SVC(),
    "Random Forest": RandomForestClassifier(),

}


In [17]:
def evaluate_models(X_train, X_test, y_train, y_test, models: dict):
    """
    Trains and evaluates multiple models.

    Parameters:
        X_train, X_test: your data splits
        y_train, y_test: your single-column numeric target variable splits (e.g., 0/1)
        models (dict): A dictionary of models e.g., {'Logistic Regression': LogisticRegression()}

    Returns:
        results_df: A DataFrame showing the scores of each model
    """
    results = []

    # y_train and y_test are already single-column Series after the change above
    # No need to select 'spam' column here anymore

    for name, model in models.items():
        # Train the model using the single-column numeric labels
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        results.append({
            'Model': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            # Use pos_label=1 since 'spam' is encoded as 1
            # Specify the labels that can appear in y_true and y_pred
            'Precision': precision_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0, labels=[0, 1]),
            'Recall': recall_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0, labels=[0, 1]),
            'F1 Score': f1_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0, labels=[0, 1])
        })

    results_df = pd.DataFrame(results).sort_values(by='F1 Score', ascending=False)
    return results_df

results_df = evaluate_models(X_train, X_test, y_train, y_test, models)
print(results_df)

                    Model  Accuracy  Precision    Recall  F1 Score
2  Support Vector Machine  0.982063   1.000000  0.865772  0.928058
3           Random Forest  0.976682   0.984252  0.838926  0.905797
0             Naive Bayes  0.972197   0.991667  0.798658  0.884758
1     Logistic Regression  0.970404   0.983333  0.791946  0.877323
